In [32]:
import pandas as pd
# import geopandas as gpd
import folium
import branca.colormap as cm
import json
import requests

In [3]:
!pip install folium

In [29]:
f = open('geojson_lnglat.json', 'r')
geojson = json.load(f)
geojson

{'type': 'FeatureCollection',
 'name': 'PARKPLANTSOENGROEN',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'id': 1,
   'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[4.826794, 52.379046],
      [4.826812, 52.379006],
      [4.82686, 52.378987],
      [4.82906, 52.37864],
      [4.829145, 52.378594],
      [4.829451, 52.378579],
      [4.829563, 52.378526],
      [4.82961, 52.378521],
      [4.829683, 52.378541],
      [4.829904, 52.378507],
      [4.829935, 52.378475],
      [4.830154, 52.378469],
      [4.830289, 52.378417],
      [4.830374, 52.378434],
      [4.832017, 52.378179],
      [4.832043, 52.378152],
      [4.832096, 52.378139],
      [4.832269, 52.37814],
      [4.832357, 52.378126],
      [4.832415, 52.378085],
      [4.832528, 52.378069],
      [4.832654, 52.37808],
      [4.832877, 52.378045],
      [4.832967, 52.377999],
      [4.833071, 52.378014],
      [4.83634, 52.377503],
      [4.83

In [27]:
filtered = [geo for geo in geojson['features'] if geo['properties']['Stadspark'] == 'J']

In [36]:
geojson['features'] = filtered
geojson

{'type': 'FeatureCollection',
 'name': 'PARKPLANTSOENGROEN',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'id': 2,
   'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[4.850505, 52.344519],
       [4.850531, 52.344504],
       [4.850831, 52.343772],
       [4.850875, 52.343605],
       [4.850876, 52.34349],
       [4.850845, 52.3434],
       [4.850752, 52.343323],
       [4.850315, 52.343193],
       [4.849829, 52.343079],
       [4.849651, 52.343067],
       [4.849584, 52.343075],
       [4.849548, 52.343089],
       [4.849505, 52.343166],
       [4.849522, 52.3433],
       [4.849767, 52.343978],
       [4.85001, 52.344445],
       [4.850191, 52.344696],
       [4.850274, 52.344747],
       [4.850344, 52.344751],
       [4.850403, 52.344716],
       [4.850505, 52.344519]]],
     [[[4.850613, 52.34499],
       [4.850399, 52.344959],
       [4.850321, 52.344998],
       [4.850276, 52.345081],
       [

In [40]:
city_parks = 'geojson_lnglat.json'

m = folium.Map(
    location=[52.377956, 4.897070],
    tiles="cartodbpositron",
    zoom_start=11,
)

folium.GeoJson(geojson, name="City Parks").add_to(m)

folium.LayerControl().add_to(m)


# var picnic_parks = L.geoJson(myJson, {filter: picnicFilter}).addTo(map);

# function picnicFilter(feature) {
#   if (feature.properties.Picnic === "Yes") return true
# }


m